In [5]:
import glob
import pandas as pd
import os

path = "../../Telemetry"
all_files = glob.glob(path + "/*.csv")

if not all_files:
    print(f"No CSV files found in {os.path.abspath(path)}. Ensure the files are placed correctly.")
else:
    combined_csv = pd.concat([pd.read_csv(file) for file in all_files])
    combined_csv.to_csv("telemetry.csv", index=False)


In [1]:
import pandas as pd

# Define chunk size (adjust based on your system's memory capacity)
chunk_size = 6

# Initialize an empty list to hold merged chunks
merged_chunks = []

# Read and preprocess the first dataset (if it’s smaller and fits in memory)
df1 = pd.read_csv("../../PitStops/Real_Pit_Stop_Data_3.csv")
df1['Lap Stop'] = pd.to_numeric(df1['Lap Stop'], errors='coerce')
df1['Processed_GrandPrix'] = df1['Grand Prix'].str.split(',').str[0].str.strip()

# Read the second dataset in chunks
for chunk in pd.read_csv("telemetry.csv", chunksize=chunk_size):
    # Preprocess the chunk
    chunk['LapNumber'] = pd.to_numeric(chunk['LapNumber'], errors='coerce')
    chunk['Processed_RaceName'] = chunk['RaceName'].str.extract(r'(.*?) Grand Prix')
    
    # Merge the chunk with the first dataset
    merged_chunk = pd.merge(
        df1,
        chunk,
        left_on=['Processed_GrandPrix', 'Lap Stop'],
        right_on=['Processed_RaceName', 'LapNumber'],
        how='inner'  # Adjust as needed (inner, left, right, or outer)
    )
    
    # Append the merged chunk to the list
    merged_chunks.append(merged_chunk)

# Concatenate all merged chunks into a single DataFrame
merged_df = pd.concat(merged_chunks, ignore_index=True)

# Drop unnecessary columns
merged_df.drop(columns=['Processed_GrandPrix', 'Processed_RaceName'], inplace=True)

# Output the final merged DataFrame
print(merged_df)


: 

: 